In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

In [ ]:
## Paths Input Here
analysis = 'cleaned_data'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/metadata/subiculum_roi_connectivity/matrix_corrMx_T (5).csv'
    clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/metadata/cog_decline_scores.csv'
    # clin_path = 'path to clinical values'
    out_name = os.path.basename(conn_path).split('.csv')[0]
    out_dir = os.path.join(os.path.dirname(conn_path), out_name)
    #out_dir = r'path to out dir here'
    x_roi_names = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/metadata/subiculum_roi_connectivity/matrix_corrMx_names (5).csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\AD_to_memory_net'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

##User Input Goes Here
x_cols = [1] #enter target data columns (indexed from 0)
#x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\spcorrel\vtafxconn-yeoseed\cta_conn_names.csv'
#roi_names = '<path to roi name location>'

## Importing Dataframe
The critical step is to develop an indication of patient ID.
The patient ID is used ot conjoin to clinical responses. 

In [ ]:
sort = False


#----------------------------------------------------------------Do not change below----------------------------------------------------------------
from calvin_utils.regex_functions import extract_digits

#Step 1) Import the data
try:
    name_df = pd.read_csv(x_roi_names, names=['arb'], header=None)
    name_df.tail(5)
    colnames = name_df.arb.values.tolist()
    newname = []
    for name in colnames:
        arb = os.path.basename(name).split('.nii')[0]
        arb = arb[0::]
        newname.append(arb)

    x_df = pd.read_csv(conn_path, names=newname, header=None)#, ignore_index=True)
    x_df.index = newname
    x_df = x_df.iloc[:,:]
    x_df = x_df.reset_index(drop=True)
    names=[]
    for name in x_df.columns.values.tolist():
        try:
            names.append(extract_digits(name)[0])
        except:
            names.append(name)
    print(names)
    x_df['Patient # CDR, ADAS'] = names
    x_df = x_df.set_index('Patient # CDR, ADAS')
except:
    print('excepted')
    x_df = pd.read_csv(conn_path)
    
display(x_df)

In [ ]:


# Step 2) Identify the column with Patient ID in it
# Gather patient IDs from data
if 'roi' in x_df.columns:
    print('roi detected')
    #Clean ROI Names
    patient_id = []
    for i in range(0, len(x_df['roi'].values.tolist())):
        roi = x_df['roi'].values.tolist()[i]
        basename = os.path.splitext(roi)[0]
        first_half = basename.split('sub-')[1]
        complete_name = first_half.split('uvat')[0]
        patient_id.append(complete_name)
    #Set names in a column
    x_df['Patient # CDR, ADAS'] = patient_id
    x_df.pop('roi')
    
elif 'Unnamed: 0' in x_df.columns:
    print('unnamed detected')
    #Intialize conainter
    try:
        patient_id = []
        cols = []
        for name in x_df['Unnamed: 0'].values.tolist():
            try:
                patient_id.append(extract_digits(name)[0])
            except:
                print('No digits in name, returning -1')
                patient_id.append('10000')
        x_df['Unnamed: 0'] = patient_id
        x_df = x_df.reset_index(drop=True)
    except:
        raise Exception('Failed at Step 2, unable to identify column with patient IDs')
    x_df['Patient # CDR, ADAS'] = x_df['Unnamed: 0']
    x_df.pop('Unnamed: 0')

#Catch when an roi-roi correl is combined with the name
elif x_df.columns.values.tolist()[0][0:3] == 'sub':
    print('Sub detected')
    #Clean ROI Names
    patient_id = []
    for i in range(0, len(x_df.columns.values.tolist())):
        roi = x_df.columns.values.tolist()[i]
        basename = os.path.splitext(roi)[0]
        first_half = basename.split('sub-')[1]
        complete_name = first_half.split('uvat')[0]
        patient_id.append(complete_name)
    #Set names in a column
    x_df['Patient # CDR, ADAS'] = patient_id
    x_df = x_df.reset_index(drop=True)
    # x_df.pop('roi')
else:
    print('Failed to detect anything')
    patient_id = []
    try:
        try:
            patient_id.append(extract_digits(x_df.columns.values.tolist()))
        except:
            print('No digits in name, returning 10000')
            patient_id.append('10000')
        x_df['Patient # CDR, ADAS'] = patient_id
    except:
        x_df['Patient # CDR, ADAS'] = x_df.columns.values
        print('Could not manipulate')
                     
#Step 3) Sort by Patient ID
if sort:
    import natsort
    #Sort the columns
    natsorted_columns = natsort.natsorted(x_df.columns)
    x_df = x_df.reindex(columns=natsorted_columns)
    
    #Set patient ID as index and sort
    try:
        x_df = x_df.drop_duplicates(subset=['Patient # CDR, ADAS'])
        x_df = x_df.set_index('Patient # CDR, ADAS', drop=True)
        natsorted_index = natsort.natsorted(x_df.index)
    except:
        # x_df = x_df.drop_duplicates(subset=['Patient # CDR, ADAS'])
        natsorted_index = natsort.natsorted(x_df.index)
    x_df = x_df.reindex(index=natsorted_index)
else:
    x_df = x_df.set_index('Patient # CDR, ADAS', drop=True)
x_df.to_csv(conn_path.split('.csv')[0] + '_revised.csv')
display(x_df)

In [ ]:
# import pandas as pd

# new_df = pd.DataFrame(index=x_df.index, columns=x_df.columns[::2])
# for i in range(0, x_df.shape[1]//2):
#     new_df.iloc[:, i] = x_df.iloc[:, i*2]

# new_df.dropna(axis=1, how='all', inplace=True)
# new_df

import pandas as pd
new_df = x_df.copy()
new_df.set_index(['Unnamed: 0'], inplace=True)
fresh_dict = {}
for name in new_df.index:
    id = name.split('_')[0]
    if name.split('_')[1][0:3] == 'vat':
        fresh_dict[id] = new_df.loc[name, f'{id}_generated_nifti.nii']
corr_df = pd.DataFrame(fresh_dict, index=['spatial_correlation']).transpose()
corr_df['Patient # CDR, ADAS'] = corr_df.index
# corr_df['Patient # CDR, ADAS'] = corr_df.index.co
display(corr_df)

In [ ]:
# new_df = pd.DataFrame()
# new_df = pd.DataFrame(np.nan, index=np.arange(50), columns=['Spatial Correlation'])
# new_df.set_index(x_df.index, inplace=True)
# new_df.shape
# for i in range(0, x_df.shape[0]):
#     new_df.iloc[i, 0] = x_df.iloc[i, i*2]
# new_df
# new_df.to_csv(os.path.join('/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_data', 'new_df.csv'))

Visualize the dataframe above and create a new dataframe with only the relevant rows and columns. 

In [ ]:
# Define columns to keep
cols_of_interest = pd.IndexSlice['memory_roi']

# Define rows to keep
rows_of_interest = pd.IndexSlice['103_vat_seed_compound_fMRI_efield':'101_vat_seed_compound_fMRI_efield']

###----------------------------------------------------------------USER INPUT----------------------------------------------------------------
t_df = x_df.copy()

# Include Rows
# cols_of_interest = t_df.columns.difference(cols_of_disinterest)

# Remove cols_of_disinterest from corr_df
# t_df.set_index('Unnamed: 0', inplace=True)
t_df = t_df.iloc[:50, :]
corr_df = t_df.loc[:, [cols_of_interest]]
# corr_df = t_df.iloc[1:, 0]
# corr_df = corr_df.rename(columns={'sub-ADAS11uAD1uverifyuvoxutstatufwep_space-2mm_lesionMask_T': 'Connectivity to ADAS Cog-11 ROI'})
corr_df.to_csv(conn_path.split('.csv')[0] + 'revised_connection_of_interest.csv')
# corr_df.rename(columns={cols_of_interest[0]: 'ADAS-Cog 11 Map'}, inplace=True)
# corr_df.reset_index(inplace=True)
display(corr_df)

In [ ]:
##--emergency code--#
# names=[]
# for name in corr_df['Patient # CDR, ADAS'].values.tolist():
#     names.append(extract_digits(name)[0])
# corr_df['Patient # CDR, ADAS'] = names
# corr_df.pop('index')
# x_df = x_df.set_index('Patient # CDR, ADAS')
display(corr_df)

In [ ]:
sort = True

#----------------------------------------------------------------User Input Above----------------------------------------------------------------
sheet_name = 'AD_Clinical_Scores'
alphab_cols = 'C, D, E, F, G, J, V'
clin_df = pd.read_excel(clin_path, sheet_name=sheet_name, usecols=alphab_cols, nrows=50)
print('Num NaNs: ', clin_df.isna().sum().sum())
# clin_df.tail(5)
if sort:
    clin_df.sort_values(by='Patient # CDR, ADAS', ascending=True, inplace=True)
    clin_df = clin_df.set_index('Patient # CDR, ADAS')

# clin_df = pd.read_csv(clin_path)
y_df = clin_df
display(y_df)

In [ ]:
corr_df

In [ ]:
#Merge and save
##--------------------------------OPTION A: Custom Merge
from calvin_utils.merge_dataframes import merge_dataframes
# data_df = merge_dataframes(y_df, corr_df)
# # data_df['Patient # CDR, ADAS'] = x_df.index.values.tolist()[0:-1]
# try:
#     data_df = data_df.drop(index=np.NaN)
# except:
#     print('No NaNs in index')
# display(data_df)
##--------------------------------OPTION B: Perform a Union on to Patient ID
# y_df['Patient # CDR, ADAS']=y_df['Patient # CDR, ADAS'].astype(int)
# corr_df.reset_index(inplace=True, drop=True)
corr_df['Patient # CDR, ADAS']=corr_df['Patient # CDR, ADAS'].astype(int)
data_df = y_df.merge(corr_df, on='Patient # CDR, ADAS', right_index=False)
try:
    data_df.pop('index')
except:
    print('nothing to pop')
try:
    data_df.pop('level_0')
except:
    print('nothing to pop')
display(data_df)
data_df.to_csv(out_dir + '_revised_with_clinical_data_dataframe.csv')
print('saved to:', out_dir)

## For TMS DATA

In [ ]:
y_df = pd.read_csv(clin_path)
y_df

In [ ]:
subs_list = y_df['ID'].to_list()
subs_list_basename = [os.path.basename(sub).split('_')[0] for sub in subs_list]
y_df['sub'] = subs_list_basename
y_df

In [ ]:
corr_df

In [ ]:
# x_df.drop(index=0, inplace=True)
corr_df = corr_df.iloc[0:-1, :]
subs_list = corr_df.index.to_list()
print(subs_list)
subs_list_basename = [os.path.basename(sub).split('MDST')[1].split('_')[0] for sub in subs_list]
corr_df['sub'] = subs_list_basename

In [ ]:
data_df = y_df.merge(corr_df, on='sub', right_index=False)
data_df

In [ ]:
data_df.to_csv(out_dir+'_all_info_with_clinical_data.csv')